# Job Hackthon Comeption

## Step 1: Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

## Step:2 Loading train Data

In [2]:
train_data = pd.read_csv("Data/train.csv")
train_data.head()

,id,gender,area,qualification,income,marital_status,vintage,claim_amount,num_policies,policy,type_of_policy,cltv
0,1,Male,Urban,Bachelor,5L-10L,1,5,5790,More than 1,A,Platinum,64308
1,2,Male,Rural,High School,5L-10L,0,8,5080,More than 1,A,Platinum,515400
2,3,Male,Urban,Bachelor,5L-10L,1,8,2599,More than 1,A,Platinum,64212
3,4,Female,Rural,High School,5L-10L,0,7,0,More than 1,A,Platinum,97920
4,5,Male,Urban,High School,More than 10L,1,6,3508,More than 1,A,Gold,59736


## Step:3 Data Pre-processing

In [3]:
cat_cols = ["gender","area","qualification","income","num_policies","policy","type_of_policy"]
## ENcoding
train_data_ready = pd.get_dummies(data=train_data, columns = cat_cols)

In [4]:
# droping id columns
train_data_ready = train_data_ready.drop("id",axis =1)
train_data_ready.head()

,marital_status,vintage,claim_amount,cltv,gender_Female,gender_Male,area_Rural,area_Urban,qualification_Bachelor,qualification_High School,...,income_<=2L,income_More than 10L,num_policies_1,num_policies_More than 1,policy_A,policy_B,policy_C,type_of_policy_Gold,type_of_policy_Platinum,type_of_policy_Silver
0,1,5,5790,64308,0,1,0,1,1,0,...,0,0,0,1,1,0,0,0,1,0
1,0,8,5080,515400,0,1,1,0,0,1,...,0,0,0,1,1,0,0,0,1,0
2,1,8,2599,64212,0,1,0,1,1,0,...,0,0,0,1,1,0,0,0,1,0
3,0,7,0,97920,1,0,1,0,0,1,...,0,0,0,1,1,0,0,0,1,0
4,1,6,3508,59736,0,1,0,1,0,1,...,0,1,0,1,1,0,0,1,0,0


In [5]:
# creating target variable and feature
x = train_data_ready.drop(["cltv"],axis = 1)
y = train_data_ready["cltv"]

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn import linear_model, tree, ensemble

In [28]:
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [29]:
cnt = 1
# split()  method generate indices to split data into training and test set.
for train_index, test_index in kf.split(x, y):
    print(f'Fold:{cnt}, Train set: {len(train_index)}, Test set:{len(test_index)}')
    cnt+=1

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


Fold:1, Train set: 80452, Test set:8940
Fold:2, Train set: 80452, Test set:8940
Fold:3, Train set: 80453, Test set:8939
Fold:4, Train set: 80453, Test set:8939
Fold:5, Train set: 80453, Test set:8939
Fold:6, Train set: 80453, Test set:8939
Fold:7, Train set: 80453, Test set:8939
Fold:8, Train set: 80453, Test set:8939
Fold:9, Train set: 80453, Test set:8939
Fold:10, Train set: 80453, Test set:8939


In [30]:
import catboost as cb
from sklearn.metrics import make_scorer, r2_score

In [31]:
r2_scorer = make_scorer(r2_score)

In [32]:
model = cb.CatBoostRegressor(boosting_type = 'Ordered', 
                             border_count = 32, 
                             depth = 4, 
                             iterations = 200, 
                             l2_leaf_reg = 3, 
                             leaf_estimation_iterations = 1, 
                             learning_rate = 0.1, 
                             loss_function = 'RMSE')

In [33]:
score = cross_val_score(model, x, y, cv= kf, scoring= r2_scorer)
print(f'Scores for each fold are: {score}')
print(f'Average score: {"{:.5f}".format(score.mean())}')

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


0:	learn: 89295.3883627	total: 15.5ms	remaining: 3.08s
1:	learn: 88203.9009863	total: 30.4ms	remaining: 3.01s
2:	learn: 87314.7332415	total: 47ms	remaining: 3.09s
3:	learn: 86582.6904907	total: 62.8ms	remaining: 3.08s
4:	learn: 85995.5189943	total: 77ms	remaining: 3s
5:	learn: 85487.2004518	total: 90.7ms	remaining: 2.93s
6:	learn: 85074.9916150	total: 104ms	remaining: 2.88s
7:	learn: 84735.0004961	total: 118ms	remaining: 2.82s
8:	learn: 84480.6552699	total: 130ms	remaining: 2.76s
9:	learn: 84237.0310079	total: 144ms	remaining: 2.73s
10:	learn: 84043.8247882	total: 157ms	remaining: 2.69s
11:	learn: 83890.4538081	total: 170ms	remaining: 2.66s
12:	learn: 83754.4838800	total: 183ms	remaining: 2.63s
13:	learn: 83644.1823818	total: 196ms	remaining: 2.61s
14:	learn: 83540.1054335	total: 210ms	remaining: 2.58s
15:	learn: 83455.7324287	total: 223ms	remaining: 2.56s
16:	learn: 83383.5788304	total: 238ms	remaining: 2.56s
17:	learn: 83327.3005776	total: 257ms	remaining: 2.6s
18:	learn: 83277.07966

153:	learn: 82800.5446163	total: 2.36s	remaining: 704ms
154:	learn: 82799.9141847	total: 2.38s	remaining: 690ms
155:	learn: 82799.4460218	total: 2.39s	remaining: 674ms
156:	learn: 82799.4218743	total: 2.4s	remaining: 658ms
157:	learn: 82798.7969804	total: 2.41s	remaining: 642ms
158:	learn: 82798.2331861	total: 2.43s	remaining: 626ms
159:	learn: 82798.1989711	total: 2.44s	remaining: 610ms
160:	learn: 82798.1757142	total: 2.45s	remaining: 594ms
161:	learn: 82797.4900872	total: 2.46s	remaining: 578ms
162:	learn: 82795.9262721	total: 2.48s	remaining: 562ms
163:	learn: 82794.4621345	total: 2.49s	remaining: 546ms
164:	learn: 82794.4377997	total: 2.5s	remaining: 530ms
165:	learn: 82793.3692528	total: 2.51s	remaining: 515ms
166:	learn: 82793.3417608	total: 2.52s	remaining: 499ms
167:	learn: 82792.2603978	total: 2.54s	remaining: 483ms
168:	learn: 82790.9188672	total: 2.55s	remaining: 468ms
169:	learn: 82790.9086206	total: 2.56s	remaining: 452ms
170:	learn: 82790.8882405	total: 2.58s	remaining: 

104:	learn: 83083.5754094	total: 1.44s	remaining: 1.3s
105:	learn: 83081.5956573	total: 1.46s	remaining: 1.29s
106:	learn: 83081.1555369	total: 1.47s	remaining: 1.28s
107:	learn: 83079.6725396	total: 1.49s	remaining: 1.27s
108:	learn: 83078.9323534	total: 1.5s	remaining: 1.25s
109:	learn: 83077.3325855	total: 1.51s	remaining: 1.24s
110:	learn: 83077.1658961	total: 1.52s	remaining: 1.22s
111:	learn: 83076.1297830	total: 1.54s	remaining: 1.21s
112:	learn: 83075.1582198	total: 1.55s	remaining: 1.19s
113:	learn: 83074.6926921	total: 1.56s	remaining: 1.18s
114:	learn: 83073.7792599	total: 1.58s	remaining: 1.17s
115:	learn: 83071.5527068	total: 1.59s	remaining: 1.15s
116:	learn: 83071.5525722	total: 1.6s	remaining: 1.14s
117:	learn: 83071.4952185	total: 1.61s	remaining: 1.12s
118:	learn: 83070.2840485	total: 1.63s	remaining: 1.11s
119:	learn: 83068.3529839	total: 1.64s	remaining: 1.09s
120:	learn: 83067.1756442	total: 1.65s	remaining: 1.08s
121:	learn: 83066.4211048	total: 1.67s	remaining: 1

58:	learn: 83016.7790454	total: 1.08s	remaining: 2.59s
59:	learn: 83014.6742595	total: 1.11s	remaining: 2.58s
60:	learn: 83013.6851074	total: 1.13s	remaining: 2.57s
61:	learn: 83011.0205257	total: 1.15s	remaining: 2.55s
62:	learn: 83009.4027541	total: 1.17s	remaining: 2.53s
63:	learn: 83008.3420398	total: 1.19s	remaining: 2.52s
64:	learn: 83006.6020944	total: 1.21s	remaining: 2.5s
65:	learn: 83005.3192383	total: 1.23s	remaining: 2.49s
66:	learn: 83004.9224761	total: 1.24s	remaining: 2.46s
67:	learn: 83002.4052003	total: 1.26s	remaining: 2.44s
68:	learn: 82999.8560020	total: 1.28s	remaining: 2.43s
69:	learn: 82999.2057150	total: 1.3s	remaining: 2.41s
70:	learn: 82997.4585268	total: 1.32s	remaining: 2.4s
71:	learn: 82996.4650416	total: 1.34s	remaining: 2.39s
72:	learn: 82993.3490842	total: 1.36s	remaining: 2.37s
73:	learn: 82990.6631061	total: 1.38s	remaining: 2.35s
74:	learn: 82986.9905777	total: 1.4s	remaining: 2.34s
75:	learn: 82984.9065487	total: 1.42s	remaining: 2.32s
76:	learn: 829

...

186:	learn: 82900.2094133	total: 3.4s	remaining: 236ms
187:	learn: 82900.2069692	total: 3.42s	remaining: 218ms
188:	learn: 82900.2068653	total: 3.43s	remaining: 200ms
189:	learn: 82900.1541112	total: 3.45s	remaining: 182ms
190:	learn: 82900.1521412	total: 3.47s	remaining: 163ms
191:	learn: 82897.3356531	total: 3.49s	remaining: 145ms
192:	learn: 82897.3355788	total: 3.5s	remaining: 127ms
193:	learn: 82897.3355186	total: 3.52s	remaining: 109ms
194:	learn: 82897.3354698	total: 3.53s	remaining: 90.5ms
195:	learn: 82897.3208664	total: 3.55s	remaining: 72.4ms
196:	learn: 82897.3041785	total: 3.56s	remaining: 54.3ms
197:	learn: 82897.2408776	total: 3.58s	remaining: 36.1ms
198:	learn: 82897.1492238	total: 3.6s	remaining: 18.1ms
199:	learn: 82897.1384094	total: 3.61s	remaining: 0us
0:	learn: 89186.3542844	total: 19.4ms	remaining: 3.86s
1:	learn: 88096.8335738	total: 37.4ms	remaining: 3.7s
2:	learn: 87194.4828192	total: 57.7ms	remaining: 3.79s
3:	learn: 86475.9517581	total: 78.7ms	remaining: 3.8

136:	learn: 82711.9181326	total: 2.59s	remaining: 1.19s
137:	learn: 82711.4006104	total: 2.61s	remaining: 1.17s
138:	learn: 82710.8632389	total: 2.63s	remaining: 1.15s
139:	learn: 82709.6245611	total: 2.65s	remaining: 1.14s
140:	learn: 82708.5480870	total: 2.67s	remaining: 1.12s
141:	learn: 82707.8340855	total: 2.69s	remaining: 1.1s
142:	learn: 82704.8282400	total: 2.7s	remaining: 1.08s
143:	learn: 82704.0554969	total: 2.72s	remaining: 1.06s
144:	learn: 82703.2472219	total: 2.74s	remaining: 1.04s
145:	learn: 82702.4925859	total: 2.76s	remaining: 1.02s
146:	learn: 82701.8810585	total: 2.78s	remaining: 1s
147:	learn: 82700.7993574	total: 2.8s	remaining: 983ms
148:	learn: 82700.4453299	total: 2.82s	remaining: 965ms
149:	learn: 82699.1198311	total: 2.84s	remaining: 946ms
150:	learn: 82698.4454494	total: 2.86s	remaining: 927ms
151:	learn: 82698.0204277	total: 2.88s	remaining: 908ms
152:	learn: 82697.5950531	total: 2.9s	remaining: 890ms
153:	learn: 82697.3993775	total: 2.92s	remaining: 871ms

88:	learn: 82958.4097589	total: 1.94s	remaining: 2.42s
89:	learn: 82957.8493017	total: 1.96s	remaining: 2.4s
90:	learn: 82956.7449406	total: 1.98s	remaining: 2.38s
91:	learn: 82955.0837640	total: 2s	remaining: 2.35s
92:	learn: 82953.9982726	total: 2.02s	remaining: 2.33s
93:	learn: 82953.4318743	total: 2.04s	remaining: 2.3s
94:	learn: 82951.4179442	total: 2.06s	remaining: 2.28s
95:	learn: 82950.6987920	total: 2.08s	remaining: 2.26s
96:	learn: 82949.0610492	total: 2.11s	remaining: 2.24s
97:	learn: 82947.8408213	total: 2.13s	remaining: 2.21s
98:	learn: 82946.4896764	total: 2.15s	remaining: 2.19s
99:	learn: 82945.2731069	total: 2.17s	remaining: 2.17s
100:	learn: 82944.2282488	total: 2.19s	remaining: 2.15s
101:	learn: 82942.2238509	total: 2.22s	remaining: 2.13s
102:	learn: 82941.2996023	total: 2.24s	remaining: 2.11s
103:	learn: 82939.2531247	total: 2.26s	remaining: 2.08s
104:	learn: 82938.6190086	total: 2.28s	remaining: 2.06s
105:	learn: 82937.2402855	total: 2.3s	remaining: 2.04s
106:	learn

42:	learn: 82938.4220289	total: 853ms	remaining: 3.12s
43:	learn: 82937.6386809	total: 870ms	remaining: 3.08s
44:	learn: 82933.4898373	total: 890ms	remaining: 3.06s
45:	learn: 82929.9722628	total: 909ms	remaining: 3.04s
46:	learn: 82924.4222133	total: 927ms	remaining: 3.02s
47:	learn: 82919.7693633	total: 946ms	remaining: 3s
48:	learn: 82915.4881924	total: 964ms	remaining: 2.97s
49:	learn: 82912.1571960	total: 983ms	remaining: 2.95s
50:	learn: 82910.7895959	total: 1s	remaining: 2.92s
51:	learn: 82907.2150997	total: 1.02s	remaining: 2.9s
52:	learn: 82903.2404491	total: 1.04s	remaining: 2.88s
53:	learn: 82901.3172045	total: 1.06s	remaining: 2.86s
54:	learn: 82900.1723164	total: 1.08s	remaining: 2.84s
55:	learn: 82898.4308006	total: 1.1s	remaining: 2.82s
56:	learn: 82895.7979997	total: 1.11s	remaining: 2.8s
57:	learn: 82892.6549290	total: 1.13s	remaining: 2.78s
58:	learn: 82891.3086718	total: 1.15s	remaining: 2.76s
59:	learn: 82887.6521827	total: 1.17s	remaining: 2.74s
60:	learn: 82884.05

196:	learn: 82774.5527729	total: 3.63s	remaining: 55.2ms
197:	learn: 82774.5493059	total: 3.65s	remaining: 36.8ms
198:	learn: 82774.5464976	total: 3.66s	remaining: 18.4ms
199:	learn: 82774.5188302	total: 3.68s	remaining: 0us
Scores for each fold are: [0.15503651 0.159996   0.16445688 0.15867213 0.16750798 0.16586495
 0.16053382 0.15773463 0.16080083 0.15932038]
Average score: 0.16099


In [53]:
from xgboost import XGBRegressor
regressor = XGBRegressor(
   colsample_bytree = 0.5, 
   learning_rate = 0.1, 
    max_depth = 3, 
    n_estimators = 100, 
    subsample =1
)

In [35]:
import re

regex = re.compile(r"\[|\]|<", re.IGNORECASE)

x.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in x.columns.values]

In [36]:
score = cross_val_score(regressor, x, y, cv= kf, scoring= r2_scorer)
print(f'Scores for each fold are: {score}')
print(f'Average score: {"{:.5f}".format(score.mean())}')

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


Scores for each fold are: [0.15477343 0.16016969 0.16319576 0.15792641 0.1684874  0.1647364
 0.16002338 0.15796513 0.1614822  0.16068632]
Average score: 0.16094


In [37]:
import lightgbm as lgb
from lightgbm import LGBMRegressor

In [38]:
gbm = lgb.LGBMRegressor(boosting_type = 'gbdt', 
                        learning_rate = 0.1, 
                        n_estimators = 100, 
                        num_leaves = 31)

In [39]:
score = cross_val_score(gbm, x, y, cv= kf, scoring= r2_scorer)
print(f'Scores for each fold are: {score}')
print(f'Average score: {"{:.5f}".format(score.mean())}')

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


Scores for each fold are: [0.15124254 0.15743209 0.15885698 0.15628713 0.16667876 0.15997811
 0.1572621  0.15338201 0.15673178 0.1595956 ]
Average score: 0.15774


In [40]:
# based on 10 k cross validation we will choose catboost as model among three trained model.

In [41]:
model = cb.CatBoostRegressor(boosting_type = 'Ordered', 
                             border_count = 32, 
                             depth = 4, 
                             iterations = 200, 
                             l2_leaf_reg = 3, 
                             leaf_estimation_iterations = 1, 
                             learning_rate = 0.1, 
                             loss_function = 'RMSE')

In [42]:
model.fit(x,y)

0:	learn: 89323.9567343	total: 17.7ms	remaining: 3.51s
1:	learn: 88244.5746754	total: 33.3ms	remaining: 3.3s
2:	learn: 87348.0432297	total: 48.6ms	remaining: 3.19s
3:	learn: 86605.8915954	total: 65.2ms	remaining: 3.19s
4:	learn: 85989.6013247	total: 79.3ms	remaining: 3.09s
5:	learn: 85501.2840232	total: 93.6ms	remaining: 3.03s
6:	learn: 85088.3934940	total: 108ms	remaining: 2.97s
7:	learn: 84763.1708418	total: 121ms	remaining: 2.91s
8:	learn: 84489.7050436	total: 136ms	remaining: 2.89s
9:	learn: 84269.1696747	total: 151ms	remaining: 2.87s
10:	learn: 84083.8416955	total: 166ms	remaining: 2.85s
11:	learn: 83915.5448404	total: 183ms	remaining: 2.86s
12:	learn: 83779.0570709	total: 198ms	remaining: 2.85s
13:	learn: 83661.5508140	total: 214ms	remaining: 2.85s
14:	learn: 83566.4846958	total: 233ms	remaining: 2.87s
15:	learn: 83488.0837851	total: 251ms	remaining: 2.89s
16:	learn: 83423.5865596	total: 267ms	remaining: 2.87s
17:	learn: 83375.6096920	total: 281ms	remaining: 2.85s
18:	learn: 8333

156:	learn: 82881.6112048	total: 2.51s	remaining: 686ms
157:	learn: 82881.6089341	total: 2.52s	remaining: 670ms
158:	learn: 82881.6070948	total: 2.54s	remaining: 654ms
159:	learn: 82881.6069553	total: 2.55s	remaining: 638ms
160:	learn: 82881.6068423	total: 2.57s	remaining: 622ms
161:	learn: 82881.5235593	total: 2.58s	remaining: 606ms
162:	learn: 82881.5220013	total: 2.6s	remaining: 590ms
163:	learn: 82881.5207394	total: 2.61s	remaining: 574ms
164:	learn: 82881.5182534	total: 2.63s	remaining: 557ms
165:	learn: 82880.9192949	total: 2.64s	remaining: 541ms
166:	learn: 82880.9185713	total: 2.66s	remaining: 525ms
167:	learn: 82880.1435176	total: 2.67s	remaining: 509ms
168:	learn: 82880.1415749	total: 2.69s	remaining: 493ms
169:	learn: 82880.1406932	total: 2.7s	remaining: 477ms
170:	learn: 82880.1399791	total: 2.72s	remaining: 461ms
171:	learn: 82880.1394006	total: 2.73s	remaining: 445ms
172:	learn: 82880.0836271	total: 2.75s	remaining: 429ms
173:	learn: 82878.9745980	total: 2.77s	remaining: 

In [43]:
### Importing Test data

# Testing Data
test_data = pd.read_csv("Data/test.csv")
test_data.head()

test_data_processed = test_data.drop("id",axis = 1)

cat_cols = ["gender","area","qualification","income","num_policies","policy","type_of_policy"]
## ENcoding
test_data_processed = pd.get_dummies(data=test_data_processed, columns = cat_cols)

In [45]:
test_data_processed.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in test_data_processed.columns.values]

In [46]:
y_test_pred = model.predict(test_data_processed)

In [47]:
submission_data = pd.read_csv("Data/sample_submission_v1.csv")
#creating the sample submission file
submission_data["cltv"] = y_test_pred
submission_data.to_csv("sample_submission_best_model.csv",index=False)

### Ensemble Model

In [61]:
cat_boost_model  = cb.CatBoostRegressor(boosting_type = 'Ordered', 
                             border_count = 32, 
                             depth = 4, 
                             iterations = 200, 
                             l2_leaf_reg = 3, 
                             leaf_estimation_iterations = 1, 
                             learning_rate = 0.1, 
                             loss_function = 'RMSE')

In [62]:
xgb_model = XGBRegressor(
   colsample_bytree = 0.5, 
   learning_rate = 0.1, 
    max_depth = 3, 
    n_estimators = 100, 
    subsample =1
)

In [63]:
LGM_model = lgb.LGBMRegressor(boosting_type = 'gbdt', 
                        learning_rate = 0.1, 
                        n_estimators = 100, 
                        num_leaves = 31)

In [64]:
from sklearn.ensemble import StackingRegressor

In [67]:
from mlxtend.regressor import StackingCVRegressor
from sklearn.model_selection import cross_val_score
import numpy as np

In [68]:
RANDOM_SEED = 42
stack = StackingCVRegressor(regressors=(cat_boost_model, LGM_model, xgb_model),
                            meta_regressor=cat_boost_model,
                            random_state=RANDOM_SEED)

In [70]:
for clf, label in zip([cat_boost_model, LGM_model, xgb_model, stack], ['catBoost', 'LGBM', 
                                                'xgb', 
                                                'StackingCVRegressor']):
    scores = cross_val_score(clf, x, y, cv=5)
    print("R^2 Score: %0.2f (+/- %0.2f) [%s]" % (
        scores.mean(), scores.std(), label))

0:	learn: 89276.7745528	total: 14.2ms	remaining: 2.82s
1:	learn: 88213.3341164	total: 27.6ms	remaining: 2.73s
2:	learn: 87314.6694506	total: 40.9ms	remaining: 2.69s
3:	learn: 86577.7181149	total: 53.8ms	remaining: 2.64s
4:	learn: 85991.6497510	total: 67.1ms	remaining: 2.62s
5:	learn: 85491.0893024	total: 80.6ms	remaining: 2.61s
6:	learn: 85070.9823319	total: 92.9ms	remaining: 2.56s
7:	learn: 84734.2878935	total: 105ms	remaining: 2.52s
8:	learn: 84466.8548675	total: 118ms	remaining: 2.51s
9:	learn: 84253.8015323	total: 132ms	remaining: 2.5s
10:	learn: 84060.8001971	total: 145ms	remaining: 2.5s
11:	learn: 83897.8202574	total: 158ms	remaining: 2.48s
12:	learn: 83756.9247836	total: 172ms	remaining: 2.47s
13:	learn: 83637.2944068	total: 188ms	remaining: 2.5s
14:	learn: 83552.6432943	total: 202ms	remaining: 2.49s
15:	learn: 83477.6188667	total: 217ms	remaining: 2.49s
16:	learn: 83416.3966689	total: 232ms	remaining: 2.5s
17:	learn: 83362.7189169	total: 249ms	remaining: 2.52s
18:	learn: 83313.

151:	learn: 82841.9839717	total: 2.14s	remaining: 676ms
152:	learn: 82840.9607657	total: 2.16s	remaining: 662ms
153:	learn: 82840.9462519	total: 2.17s	remaining: 647ms
154:	learn: 82839.4830081	total: 2.18s	remaining: 634ms
155:	learn: 82838.5641568	total: 2.2s	remaining: 620ms
156:	learn: 82837.4430436	total: 2.21s	remaining: 606ms
157:	learn: 82836.0056541	total: 2.23s	remaining: 592ms
158:	learn: 82835.9847121	total: 2.23s	remaining: 576ms
159:	learn: 82835.9825045	total: 2.24s	remaining: 561ms
160:	learn: 82835.9536929	total: 2.25s	remaining: 546ms
161:	learn: 82835.6979115	total: 2.27s	remaining: 531ms
162:	learn: 82835.6873842	total: 2.27s	remaining: 517ms
163:	learn: 82835.6858710	total: 2.29s	remaining: 502ms
164:	learn: 82834.8565229	total: 2.3s	remaining: 487ms
165:	learn: 82834.0635476	total: 2.31s	remaining: 473ms
166:	learn: 82832.1881026	total: 2.32s	remaining: 459ms
167:	learn: 82832.1724828	total: 2.33s	remaining: 444ms
168:	learn: 82832.1598296	total: 2.34s	remaining: 

102:	learn: 82266.4450064	total: 1.7s	remaining: 1.6s
103:	learn: 82266.4108946	total: 1.71s	remaining: 1.58s
104:	learn: 82264.1763025	total: 1.73s	remaining: 1.56s
105:	learn: 82263.4948147	total: 1.74s	remaining: 1.54s
106:	learn: 82262.2765551	total: 1.75s	remaining: 1.52s
107:	learn: 82260.4766748	total: 1.77s	remaining: 1.5s
108:	learn: 82258.8523002	total: 1.78s	remaining: 1.49s
109:	learn: 82257.9498002	total: 1.79s	remaining: 1.47s
110:	learn: 82257.9336803	total: 1.8s	remaining: 1.44s
111:	learn: 82255.7583312	total: 1.81s	remaining: 1.43s
112:	learn: 82253.5276386	total: 1.83s	remaining: 1.41s
113:	learn: 82251.3210955	total: 1.84s	remaining: 1.39s
114:	learn: 82248.4431927	total: 1.85s	remaining: 1.37s
115:	learn: 82247.4168049	total: 1.86s	remaining: 1.35s
116:	learn: 82247.3249990	total: 1.88s	remaining: 1.33s
117:	learn: 82246.9348507	total: 1.89s	remaining: 1.31s
118:	learn: 82245.9941181	total: 1.9s	remaining: 1.29s
119:	learn: 82244.2091338	total: 1.92s	remaining: 1.2

52:	learn: 83105.6653255	total: 888ms	remaining: 2.46s
53:	learn: 83101.8107515	total: 902ms	remaining: 2.44s
54:	learn: 83098.8139550	total: 917ms	remaining: 2.42s
55:	learn: 83095.3765223	total: 931ms	remaining: 2.39s
56:	learn: 83093.3641198	total: 944ms	remaining: 2.37s
57:	learn: 83091.8820863	total: 957ms	remaining: 2.34s
58:	learn: 83089.0053435	total: 970ms	remaining: 2.32s
59:	learn: 83085.2373102	total: 982ms	remaining: 2.29s
60:	learn: 83082.5195673	total: 995ms	remaining: 2.27s
61:	learn: 83080.0332500	total: 1.01s	remaining: 2.24s
62:	learn: 83079.1743859	total: 1.02s	remaining: 2.21s
63:	learn: 83077.5948840	total: 1.03s	remaining: 2.19s
64:	learn: 83075.3192440	total: 1.04s	remaining: 2.17s
65:	learn: 83074.5686416	total: 1.05s	remaining: 2.14s
66:	learn: 83071.4638625	total: 1.07s	remaining: 2.13s
67:	learn: 83068.4826617	total: 1.09s	remaining: 2.11s
68:	learn: 83067.1259773	total: 1.1s	remaining: 2.09s
69:	learn: 83065.0710101	total: 1.11s	remaining: 2.07s
70:	learn: 

1:	learn: 88392.4339495	total: 34.7ms	remaining: 3.43s
2:	learn: 87501.6069842	total: 52.1ms	remaining: 3.42s
3:	learn: 86753.3424031	total: 65.7ms	remaining: 3.22s
4:	learn: 86164.8370870	total: 81.2ms	remaining: 3.16s
5:	learn: 85651.4236716	total: 97.2ms	remaining: 3.14s
6:	learn: 85244.3634694	total: 113ms	remaining: 3.11s
7:	learn: 84919.3475416	total: 126ms	remaining: 3.03s
8:	learn: 84633.1889324	total: 140ms	remaining: 2.97s
9:	learn: 84393.7188806	total: 155ms	remaining: 2.95s
10:	learn: 84212.3423425	total: 170ms	remaining: 2.92s
11:	learn: 84056.1273606	total: 185ms	remaining: 2.89s
12:	learn: 83916.1923509	total: 198ms	remaining: 2.85s
13:	learn: 83812.9656657	total: 212ms	remaining: 2.82s
14:	learn: 83717.8917698	total: 225ms	remaining: 2.78s
15:	learn: 83634.2379855	total: 239ms	remaining: 2.75s
16:	learn: 83569.3299591	total: 255ms	remaining: 2.74s
17:	learn: 83509.0545348	total: 270ms	remaining: 2.73s
18:	learn: 83458.8881831	total: 283ms	remaining: 2.7s
19:	learn: 8341

...

76:	learn: 83418.6416915	total: 1.32s	remaining: 2.11s
77:	learn: 83414.8177684	total: 1.34s	remaining: 2.09s
78:	learn: 83412.9309289	total: 1.36s	remaining: 2.08s
79:	learn: 83410.7905980	total: 1.38s	remaining: 2.06s
80:	learn: 83409.2449688	total: 1.39s	remaining: 2.05s
81:	learn: 83408.2201461	total: 1.41s	remaining: 2.03s
82:	learn: 83406.5104013	total: 1.43s	remaining: 2.02s
83:	learn: 83404.3678427	total: 1.45s	remaining: 2s
84:	learn: 83403.2846161	total: 1.47s	remaining: 1.99s
85:	learn: 83402.3305296	total: 1.49s	remaining: 1.97s
86:	learn: 83400.4130296	total: 1.5s	remaining: 1.96s
87:	learn: 83397.7013236	total: 1.52s	remaining: 1.94s
88:	learn: 83395.8510981	total: 1.54s	remaining: 1.92s
89:	learn: 83394.5319603	total: 1.56s	remaining: 1.9s
90:	learn: 83393.2834097	total: 1.57s	remaining: 1.89s
91:	learn: 83392.4847549	total: 1.59s	remaining: 1.87s
92:	learn: 83391.0067856	total: 1.61s	remaining: 1.85s
93:	learn: 83390.1107442	total: 1.63s	remaining: 1.84s
94:	learn: 8338

34:	learn: 83269.8380277	total: 664ms	remaining: 3.13s
35:	learn: 83262.1500130	total: 683ms	remaining: 3.11s
36:	learn: 83256.5655581	total: 701ms	remaining: 3.09s
37:	learn: 83253.2665230	total: 720ms	remaining: 3.07s
38:	learn: 83245.6001504	total: 741ms	remaining: 3.06s
39:	learn: 83244.4303335	total: 753ms	remaining: 3.01s
40:	learn: 83240.9429123	total: 769ms	remaining: 2.98s
41:	learn: 83236.9376095	total: 787ms	remaining: 2.96s
42:	learn: 83231.4719598	total: 805ms	remaining: 2.94s
43:	learn: 83230.5461600	total: 819ms	remaining: 2.9s
44:	learn: 83226.6997440	total: 836ms	remaining: 2.88s
45:	learn: 83222.8282037	total: 852ms	remaining: 2.85s
46:	learn: 83219.5372145	total: 869ms	remaining: 2.83s
47:	learn: 83215.8833260	total: 887ms	remaining: 2.81s
48:	learn: 83211.7905065	total: 904ms	remaining: 2.79s
49:	learn: 83209.3867663	total: 921ms	remaining: 2.76s
50:	learn: 83207.3412213	total: 937ms	remaining: 2.74s
51:	learn: 83202.5189612	total: 956ms	remaining: 2.72s
52:	learn: 

189:	learn: 83025.3209841	total: 3.44s	remaining: 181ms
190:	learn: 83025.2614598	total: 3.46s	remaining: 163ms
191:	learn: 83025.2451727	total: 3.47s	remaining: 145ms
192:	learn: 83024.7434783	total: 3.49s	remaining: 127ms
193:	learn: 83023.8240706	total: 3.51s	remaining: 109ms
194:	learn: 83023.8208942	total: 3.52s	remaining: 90.3ms
195:	learn: 83021.2031346	total: 3.54s	remaining: 72.2ms
196:	learn: 83021.0939596	total: 3.55s	remaining: 54.1ms
197:	learn: 83019.7799767	total: 3.57s	remaining: 36.1ms
198:	learn: 83019.4543814	total: 3.59s	remaining: 18ms
199:	learn: 83019.4485948	total: 3.6s	remaining: 0us
0:	learn: 89547.2606896	total: 18.7ms	remaining: 3.72s
1:	learn: 88480.3204598	total: 37ms	remaining: 3.66s
2:	learn: 87580.8575999	total: 53.5ms	remaining: 3.51s
3:	learn: 86849.2252719	total: 70.2ms	remaining: 3.44s
4:	learn: 86233.6772620	total: 86.1ms	remaining: 3.36s
5:	learn: 85728.7430025	total: 104ms	remaining: 3.36s
6:	learn: 85319.1309925	total: 120ms	remaining: 3.31s
7:	

139:	learn: 83364.2054643	total: 2.35s	remaining: 1.01s
140:	learn: 83364.2037985	total: 2.37s	remaining: 990ms
141:	learn: 83364.1417133	total: 2.38s	remaining: 972ms
142:	learn: 83364.0887198	total: 2.4s	remaining: 955ms
143:	learn: 83363.9075449	total: 2.41s	remaining: 938ms
144:	learn: 83363.8657181	total: 2.43s	remaining: 920ms
145:	learn: 83363.5269153	total: 2.44s	remaining: 904ms
146:	learn: 83362.7129696	total: 2.46s	remaining: 887ms
147:	learn: 83362.6908797	total: 2.47s	remaining: 869ms
148:	learn: 83362.3826597	total: 2.49s	remaining: 852ms
149:	learn: 83361.3338426	total: 2.51s	remaining: 836ms
150:	learn: 83361.1875170	total: 2.52s	remaining: 818ms
151:	learn: 83360.6643601	total: 2.54s	remaining: 801ms
152:	learn: 83360.4969601	total: 2.55s	remaining: 785ms
153:	learn: 83360.4311660	total: 2.57s	remaining: 768ms
154:	learn: 83360.2819664	total: 2.59s	remaining: 751ms
155:	learn: 83359.7178187	total: 2.6s	remaining: 734ms
156:	learn: 83359.3104093	total: 2.62s	remaining: 

89:	learn: 83298.0525524	total: 1.64s	remaining: 2s
90:	learn: 83297.6666451	total: 1.66s	remaining: 1.99s
91:	learn: 83295.1319937	total: 1.67s	remaining: 1.97s
92:	learn: 83294.1043497	total: 1.69s	remaining: 1.95s
93:	learn: 83293.0638792	total: 1.71s	remaining: 1.93s
94:	learn: 83291.3925867	total: 1.73s	remaining: 1.91s
95:	learn: 83290.4002480	total: 1.74s	remaining: 1.89s
96:	learn: 83288.7277521	total: 1.76s	remaining: 1.87s
97:	learn: 83287.4433746	total: 1.77s	remaining: 1.85s
98:	learn: 83286.6671914	total: 1.79s	remaining: 1.83s
99:	learn: 83284.9792662	total: 1.81s	remaining: 1.81s
100:	learn: 83283.3017285	total: 1.82s	remaining: 1.79s
101:	learn: 83282.3951538	total: 1.84s	remaining: 1.77s
102:	learn: 83280.7672121	total: 1.86s	remaining: 1.75s
103:	learn: 83279.5220958	total: 1.88s	remaining: 1.73s
104:	learn: 83278.6284027	total: 1.89s	remaining: 1.71s
105:	learn: 83276.6297624	total: 1.91s	remaining: 1.69s
106:	learn: 83275.9751333	total: 1.93s	remaining: 1.67s
107:	l

In [75]:
stack = StackingCVRegressor(regressors=(cat_boost_model, xgb_model),
                            meta_regressor=cat_boost_model,
                            random_state=RANDOM_SEED)

In [76]:
stack.fit(x,y)

0:	learn: 89592.6208523	total: 36.5ms	remaining: 7.27s
1:	learn: 88526.9414859	total: 70.2ms	remaining: 6.95s
2:	learn: 87655.1375567	total: 104ms	remaining: 6.85s
3:	learn: 86922.2129250	total: 124ms	remaining: 6.08s
4:	learn: 86330.1435853	total: 140ms	remaining: 5.45s
5:	learn: 85813.4503112	total: 155ms	remaining: 5s
6:	learn: 85389.8667956	total: 170ms	remaining: 4.7s
7:	learn: 85050.8087449	total: 185ms	remaining: 4.44s
8:	learn: 84779.8430343	total: 200ms	remaining: 4.25s
9:	learn: 84566.3091951	total: 214ms	remaining: 4.07s
10:	learn: 84373.2352491	total: 230ms	remaining: 3.96s
11:	learn: 84203.7408637	total: 250ms	remaining: 3.91s
12:	learn: 84062.0316192	total: 272ms	remaining: 3.92s
13:	learn: 83937.8980722	total: 291ms	remaining: 3.87s
14:	learn: 83841.5017165	total: 309ms	remaining: 3.81s
15:	learn: 83762.9551503	total: 327ms	remaining: 3.75s
16:	learn: 83688.7151270	total: 343ms	remaining: 3.69s
17:	learn: 83631.7324840	total: 361ms	remaining: 3.65s
18:	learn: 83584.77234

158:	learn: 83105.5017907	total: 2.58s	remaining: 665ms
159:	learn: 83105.0380821	total: 2.6s	remaining: 650ms
160:	learn: 83105.0158074	total: 2.61s	remaining: 633ms
161:	learn: 83104.9648593	total: 2.63s	remaining: 616ms
162:	learn: 83104.9466801	total: 2.64s	remaining: 600ms
163:	learn: 83104.9449800	total: 2.65s	remaining: 583ms
164:	learn: 83104.9264233	total: 2.67s	remaining: 566ms
165:	learn: 83103.5780070	total: 2.69s	remaining: 550ms
166:	learn: 83103.5643287	total: 2.7s	remaining: 534ms
167:	learn: 83102.2100419	total: 2.71s	remaining: 517ms
168:	learn: 83102.1989726	total: 2.73s	remaining: 501ms
169:	learn: 83102.1151474	total: 2.74s	remaining: 484ms
170:	learn: 83102.0738623	total: 2.75s	remaining: 467ms
171:	learn: 83099.2082420	total: 2.77s	remaining: 451ms
172:	learn: 83098.7328583	total: 2.78s	remaining: 435ms
173:	learn: 83098.7328315	total: 2.8s	remaining: 418ms
174:	learn: 83095.8627627	total: 2.81s	remaining: 402ms
175:	learn: 83095.8279588	total: 2.83s	remaining: 3

112:	learn: 83050.0705647	total: 2.18s	remaining: 1.68s
113:	learn: 83049.1826534	total: 2.2s	remaining: 1.66s
114:	learn: 83047.6849381	total: 2.23s	remaining: 1.65s
115:	learn: 83046.0896327	total: 2.25s	remaining: 1.63s
116:	learn: 83045.3239899	total: 2.27s	remaining: 1.61s
117:	learn: 83044.2275333	total: 2.29s	remaining: 1.59s
118:	learn: 83043.4933078	total: 2.31s	remaining: 1.57s
119:	learn: 83043.4835905	total: 2.33s	remaining: 1.55s
120:	learn: 83043.4660554	total: 2.34s	remaining: 1.53s
121:	learn: 83041.6290155	total: 2.36s	remaining: 1.51s
122:	learn: 83041.5819810	total: 2.37s	remaining: 1.48s
123:	learn: 83040.7370959	total: 2.39s	remaining: 1.46s
124:	learn: 83040.0342440	total: 2.4s	remaining: 1.44s
125:	learn: 83039.6503923	total: 2.42s	remaining: 1.42s
126:	learn: 83037.7483259	total: 2.44s	remaining: 1.4s
127:	learn: 83037.5675240	total: 2.46s	remaining: 1.38s
128:	learn: 83035.3425472	total: 2.48s	remaining: 1.37s
129:	learn: 83035.0492774	total: 2.5s	remaining: 1.

66:	learn: 82633.0592765	total: 1.31s	remaining: 2.6s
67:	learn: 82631.7002509	total: 1.33s	remaining: 2.58s
68:	learn: 82631.5016461	total: 1.35s	remaining: 2.56s
69:	learn: 82629.1584649	total: 1.36s	remaining: 2.53s
70:	learn: 82628.7983071	total: 1.37s	remaining: 2.5s
71:	learn: 82627.3045332	total: 1.39s	remaining: 2.47s
72:	learn: 82625.7610396	total: 1.41s	remaining: 2.45s
73:	learn: 82623.1956355	total: 1.42s	remaining: 2.42s
74:	learn: 82621.1331973	total: 1.44s	remaining: 2.39s
75:	learn: 82619.9708645	total: 1.45s	remaining: 2.37s
76:	learn: 82618.7738248	total: 1.47s	remaining: 2.34s
77:	learn: 82616.6950192	total: 1.48s	remaining: 2.32s
78:	learn: 82616.3923446	total: 1.49s	remaining: 2.29s
79:	learn: 82614.5730537	total: 1.51s	remaining: 2.27s
80:	learn: 82614.2387543	total: 1.53s	remaining: 2.24s
81:	learn: 82613.7890121	total: 1.54s	remaining: 2.22s
82:	learn: 82613.7826792	total: 1.55s	remaining: 2.19s
83:	learn: 82610.8187560	total: 1.57s	remaining: 2.17s
84:	learn: 8

20:	learn: 83303.2250387	total: 449ms	remaining: 3.82s
21:	learn: 83269.1704678	total: 468ms	remaining: 3.79s
22:	learn: 83239.4149551	total: 491ms	remaining: 3.78s
23:	learn: 83217.4674150	total: 508ms	remaining: 3.73s
24:	learn: 83197.0173319	total: 524ms	remaining: 3.67s
25:	learn: 83179.7997300	total: 539ms	remaining: 3.6s
26:	learn: 83164.3306246	total: 554ms	remaining: 3.55s
27:	learn: 83152.1666880	total: 571ms	remaining: 3.5s
28:	learn: 83142.0278988	total: 585ms	remaining: 3.45s
29:	learn: 83136.6747390	total: 598ms	remaining: 3.39s
30:	learn: 83126.6370284	total: 613ms	remaining: 3.34s
31:	learn: 83119.2749845	total: 629ms	remaining: 3.3s
32:	learn: 83108.8961339	total: 646ms	remaining: 3.27s
33:	learn: 83102.8098336	total: 664ms	remaining: 3.24s
34:	learn: 83094.5221251	total: 680ms	remaining: 3.2s
35:	learn: 83087.7397713	total: 694ms	remaining: 3.16s
36:	learn: 83082.3759886	total: 709ms	remaining: 3.12s
37:	learn: 83078.8833598	total: 724ms	remaining: 3.09s
38:	learn: 830

178:	learn: 82857.1847194	total: 3.85s	remaining: 452ms
179:	learn: 82857.1756409	total: 3.87s	remaining: 430ms
180:	learn: 82856.5847105	total: 3.89s	remaining: 409ms
181:	learn: 82855.9531458	total: 3.92s	remaining: 388ms
182:	learn: 82855.2421363	total: 3.94s	remaining: 366ms
183:	learn: 82853.0984820	total: 3.96s	remaining: 345ms
184:	learn: 82853.0482417	total: 3.99s	remaining: 323ms
185:	learn: 82853.0450958	total: 4s	remaining: 301ms
186:	learn: 82853.0030294	total: 4.02s	remaining: 280ms
187:	learn: 82852.1468694	total: 4.04s	remaining: 258ms
188:	learn: 82851.9738694	total: 4.06s	remaining: 236ms
189:	learn: 82850.7900831	total: 4.08s	remaining: 215ms
190:	learn: 82850.7832064	total: 4.09s	remaining: 193ms
191:	learn: 82848.8699570	total: 4.11s	remaining: 171ms
192:	learn: 82848.8674425	total: 4.12s	remaining: 150ms
193:	learn: 82847.1714107	total: 4.14s	remaining: 128ms
194:	learn: 82847.1092373	total: 4.16s	remaining: 107ms
195:	learn: 82847.1062662	total: 4.17s	remaining: 8

136:	learn: 82682.0502451	total: 2.33s	remaining: 1.07s
137:	learn: 82682.0498145	total: 2.35s	remaining: 1.05s
138:	learn: 82680.9064852	total: 2.36s	remaining: 1.04s
139:	learn: 82680.8457715	total: 2.38s	remaining: 1.02s
140:	learn: 82680.3985108	total: 2.4s	remaining: 1s
141:	learn: 82680.3977142	total: 2.41s	remaining: 985ms
142:	learn: 82680.2959310	total: 2.43s	remaining: 968ms
143:	learn: 82678.5578094	total: 2.45s	remaining: 952ms
144:	learn: 82678.4606605	total: 2.46s	remaining: 933ms
145:	learn: 82678.4048431	total: 2.47s	remaining: 915ms
146:	learn: 82677.4906569	total: 2.49s	remaining: 899ms
147:	learn: 82676.5301059	total: 2.51s	remaining: 882ms
148:	learn: 82676.5223384	total: 2.52s	remaining: 864ms
149:	learn: 82676.4811700	total: 2.54s	remaining: 847ms
150:	learn: 82676.4745352	total: 2.55s	remaining: 829ms
151:	learn: 82676.4408457	total: 2.57s	remaining: 811ms
152:	learn: 82675.1793798	total: 2.59s	remaining: 795ms
153:	learn: 82674.1089413	total: 2.61s	remaining: 77

95:	learn: 83006.2507682	total: 1.72s	remaining: 1.87s
96:	learn: 83005.9728502	total: 1.74s	remaining: 1.85s
97:	learn: 83005.9578891	total: 1.75s	remaining: 1.83s
98:	learn: 83005.8288444	total: 1.77s	remaining: 1.81s
99:	learn: 83005.6646547	total: 1.79s	remaining: 1.79s
100:	learn: 83005.5502609	total: 1.81s	remaining: 1.77s
101:	learn: 83005.4644487	total: 1.82s	remaining: 1.75s
102:	learn: 83005.1505109	total: 1.84s	remaining: 1.74s
103:	learn: 83004.9545714	total: 1.86s	remaining: 1.72s
104:	learn: 83004.9381374	total: 1.88s	remaining: 1.7s
105:	learn: 83004.7367491	total: 1.89s	remaining: 1.68s
106:	learn: 83004.6368039	total: 1.91s	remaining: 1.66s
107:	learn: 83004.6171084	total: 1.92s	remaining: 1.64s
108:	learn: 83004.5813564	total: 1.94s	remaining: 1.62s
109:	learn: 83004.3728673	total: 1.96s	remaining: 1.6s
110:	learn: 83004.3433339	total: 1.97s	remaining: 1.58s
111:	learn: 83004.0770566	total: 1.99s	remaining: 1.56s
112:	learn: 83004.0354611	total: 2s	remaining: 1.54s
11

46:	learn: 83013.6037202	total: 1.08s	remaining: 3.52s
47:	learn: 83010.3331746	total: 1.1s	remaining: 3.49s
48:	learn: 83006.3074743	total: 1.12s	remaining: 3.45s
49:	learn: 83005.1505418	total: 1.14s	remaining: 3.41s
50:	learn: 83003.6819161	total: 1.16s	remaining: 3.38s
51:	learn: 83003.2387245	total: 1.18s	remaining: 3.35s
52:	learn: 83001.7988733	total: 1.2s	remaining: 3.32s
53:	learn: 82998.2481678	total: 1.22s	remaining: 3.29s
54:	learn: 82997.7118724	total: 1.24s	remaining: 3.26s
55:	learn: 82993.8902093	total: 1.26s	remaining: 3.23s
56:	learn: 82991.3232286	total: 1.28s	remaining: 3.21s
57:	learn: 82988.4771808	total: 1.3s	remaining: 3.19s
58:	learn: 82984.2443061	total: 1.33s	remaining: 3.17s
59:	learn: 82980.3124423	total: 1.35s	remaining: 3.15s
60:	learn: 82977.6356290	total: 1.37s	remaining: 3.13s
61:	learn: 82975.2907148	total: 1.4s	remaining: 3.11s
62:	learn: 82972.3019569	total: 1.42s	remaining: 3.08s
63:	learn: 82970.0341771	total: 1.44s	remaining: 3.05s
64:	learn: 829

StackingCVRegressor(meta_regressor=<catboost.core.CatBoostRegressor object at 0x00000236B141E730>,
                    random_state=42,
                    regressors=(<catboost.core.CatBoostRegressor object at 0x00000236B141E730>,
                                XGBRegressor(base_score=None, booster=None,
                                             callbacks=None,
                                             colsample_bylevel=None,
                                             colsample_bynode=None,
                                             colsample_bytree=0.5,
                                             early_stopping_rounds=None,
                                             enable_categorica...
                                             gpu_id=None, grow_policy=None,
                                             importance_type=None,
                                             interaction_constraints=None,
                                             learning_rate=0.1, max_bi

In [77]:
y_test_pred = stack.predict(test_data_processed)

In [78]:
submission_data = pd.read_csv("Data/sample_submission_v1.csv")
#creating the sample submission file
submission_data["cltv"] = y_test_pred
submission_data.to_csv("sample_submission_test_20.csv",index=False)